## Классификация комментариев на позитивные и негативные

### Цель проекта:

Научить модель классификации комментариев на положительные и отрицательные. Опираемся на данные с разметкой о токсичности правок.


### Шаги выполнения проекта:

1. Предобработка данных.
2. Обучение моделей и проверка на тестовой выборке.

## 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import time
import torch
import transformers
import tensorflow as tf

In [2]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from tqdm import notebook

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem import WordNetLemmatizer 

In [4]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
toxic = pd.read_csv('/datasets/toxic_comments.csv')
toxic.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
toxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Проверим на дубли:

In [8]:
toxic.duplicated().sum()

0

Перепроверим уникальные значения в столбце с целевым признаком:

In [9]:
toxic.toxic.unique()

array([0, 1])

Оценим частоту употребления слов: для этого используем TF-IDF. Поскольку в данном случае не обойтись без упрощения текста, очистим текст от любых других знаков, кроме букв, проведем лемматизацию и содадим новый столбец: 

In [10]:
corpus = list(toxic['text'])

def lemmatize(text):      # лемматизируем текст и объединим элементы списка в строку, которую получим на выходе
    lemmatizer = WordNetLemmatizer()
    lemm_text = []
    for row in text: 
        word_list = nltk.word_tokenize(row)
        lemmas = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        lemm_text.append(lemmas)
    return (pd.Series(lemm_text))

def clear_text(text):     # очистим строки при помощи регулярных выражений: оставим только буквы.
    new_text = (re.sub(r'[^a-zA-Z]', ' ', text)).lower()
    return " ".join(new_text.split())

In [11]:
%%time
clear_lst = []
for i in corpus:
    clear_lst.append(clear_text(i))
clear_lst[:5]

CPU times: user 5.45 s, sys: 85.2 ms, total: 5.54 s
Wall time: 5.54 s


['explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now',
 'd aww he matches this background colour i m seemingly stuck with thanks talk january utc',
 'hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info',
 'more i can t make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there ap

In [12]:
%%time
toxic['lemm_text'] = lemmatize(clear_lst)
toxic.head()

CPU times: user 1min 58s, sys: 692 ms, total: 1min 59s
Wall time: 1min 59s


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [14]:
type(toxic.lemm_text[1])

str

Поскольку в дальнейшем мы будем использовать кросс-валидацию, поделим выборку на обучающую и тестовую и выведем целевой признак из датафрейма:

In [15]:
features = toxic['lemm_text']
target = toxic['toxic']

In [16]:
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                               test_size=0.1, random_state=12345)

In [17]:
print(features_train.shape, '\n', target_train.shape, '\n', features_test.shape, '\n', target_test.shape)

(143613,) 
 (143613,) 
 (15958,) 
 (15958,)


Далее преобразуем слова в векторы и оценим важность слов, но сначала перезапишем в переменные "corpus_..." уже обработанные строки, находящиеся в столбце "lemm_text" тренировочной, валидационной и тестовой выборки:

In [18]:
corpus_train = list(features_train) 
corpus_test = list(features_test) 

Чтобы почистить мешок слов, найдём стоп-слова:

In [19]:
stop_words = set(stopwords.words('english'))

Оценим важность слов: определим, как часто уникальное слово встречается во всём корпусе и в отдельном его тексте:

In [20]:
voc = TfidfVectorizer(stop_words=stop_words).fit(corpus_train)

features_train = voc.transform(corpus_train)

features_train.shape

(143613, 149248)

In [21]:
features_test = voc.transform(corpus_test)
features_test.shape

(15958, 149248)

Теперь сделаем предобработку данных, формируя векторы моделью BERT, и оценим, какой же вариант подготовки данных даст наилучший результат в обучении моделей.

При этом, чтобы не нагружать ядро и не создавать эмбеддинги слишком долго, возьмем из выборки лишь часть случайных элементов. Размер батча также зафиксируем.

In [21]:
tox = pd.read_csv('/datasets/toxic_comments.csv')
tox['text'] = clear_lst
tox.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [22]:
tox = tox.sample(3000).reset_index(drop=True)
print(tox.shape)
tox.head()

(3000, 2)


,text,toxic
0,anti north east india editor,0
1,hello again i want to read scientific answers ...,0
2,indeed so were you to delete everything from i...,0
3,telephone numbers of guest houses and hotels t...,0
4,dolerite theegyptian doleritte is also a stone,0


Инициализируем токенизатор как объект класса BertTokenizer():

In [23]:
tokenizer = transformers.BertTokenizer(
    vocab_file='vocab_deep_pavlov.txt') 

In [24]:
%%time
tokenized = tox['text'].apply(
  lambda x: tokenizer.encode(x[:512], add_special_tokens=True))
print(tokenized.head())
tokenized.shape

0        [101, 2848, 1564, 1746, 2222, 991, 3045, 102]
1    [101, 19082, 1104, 178, 880, 654, 1191, 3812, ...
2    [101, 5750, 745, 1015, 661, 654, 17596, 1917, ...
3    [101, 7314, 2849, 669, 3648, 2725, 662, 10723,...
4    [101, 722, 2879, 887, 20021, 28308, 1343, 722,...
Name: text, dtype: object
CPU times: user 3.92 s, sys: 8.05 ms, total: 3.93 s
Wall time: 3.94 s


(3000,)

Применим padding (уравняем длины исходных твитов):

In [25]:
n = max(map(len, tokenized))
n

189

In [26]:
for i in range(len(tokenized)):
    tokenized[i] = tokenized[i] + [0]*(n - len(tokenized[i]))

Создадим маску (укажем нулевые и не нулевые значения):

In [27]:
tokenized = np.stack(tokenized)
attention_mask = np.where(tokenized != 0, 1, 0)
print(attention_mask.shape)

(3000, 189)


В качестве аргумента конфигурации передадим JSON-файл с настройками "bert-base-cased" Deep Pavlov. При инициализации самой модели класса BertModel передадим ей файл с предобученной моделью (также Deep Pavlov) и конфигурацией:

In [28]:
config = transformers.BertConfig.from_json_file(
    'bert-base-cased-deep-pavlov-config.json')
model = transformers.BertModel.from_pretrained(
    'bert-base-cased-conversational.bin', config=config)

Как уже отмечалось выше, чтобы сократить объем используемой оперативной памяти, зафиксируем относительно небольшой размер батча:

In [29]:
batch_size = 100

Теперь прогоняем цикл по батчам. Для отображения процесса используем notebook():

*Внимание!* Следующая ячейка выполняется больше 1 часа.

In [30]:
%%time

embeddings = []
for i in notebook.tqdm(range(tokenized.shape[0] // batch_size)): 
    batch = torch.LongTensor(tokenized[batch_size*i:batch_size*(i+1)])   # преобразуем данные в формат тензоров 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]) # преобразуем маску

    with torch.no_grad(): # градиенты не нужны: модель BERT обучать не будем
        batch_embeddings = model(batch, attention_mask=attention_mask_batch) 
        # получим эмбеддинги для батча, передав модели данные и маску

    embeddings.append(batch_embeddings[0][:,0,:].numpy()) 
#     извлекаем нужные элементы из тензора и добавляем в список эмбеддингов


CPU times: user 1h 2min 48s, sys: 10min 42s, total: 1h 13min 31s
Wall time: 1h 13min 35s


Признаки готовы:

In [31]:
feat_bert = np.concatenate(embeddings)
feat_bert

array([[ 0.327846  ,  0.32251823, -0.08402615, ..., -0.04934282,
         0.22287056,  0.42345887],
       [ 0.83862114, -0.24543828,  0.49166483, ...,  0.5562003 ,
        -0.6390467 , -0.43757027],
       [ 0.4755483 , -0.6469525 ,  0.21635166, ...,  0.4921921 ,
        -0.5032968 , -0.27687022],
       ...,
       [ 0.4551098 , -0.412283  ,  0.46406507, ...,  0.41547883,
        -0.654666  ,  0.04324127],
       [ 0.6391937 , -0.3299532 ,  0.00486617, ...,  0.24630123,
        -0.730621  , -0.79904044],
       [ 0.52958167, -0.5483387 ,  0.2200931 , ...,  0.5129085 ,
        -0.9058263 ,  0.01320808]], dtype=float32)

Теперь разделим на выборки уже эти данные:

In [32]:
features_tr, features_val, target_tr, target_val = train_test_split(feat_bert, tox['toxic'],
                                                                               test_size=0.2, random_state=12345)

In [33]:
features_val, features_tt, target_val, target_tt = train_test_split(features_val, target_val,
                                                                               test_size=0.5, random_state=12345)

In [34]:
print(features_tr.shape, '\n', target_tr.shape, '\n', features_val.shape, '\n', target_val.shape,
      '\n', features_tt.shape, '\n', target_tt.shape)

(2400, 768) 
 (2400,) 
 (300, 768) 
 (300,) 
 (300, 768) 
 (300,)


Теперь обучим модели и выберем лучшую:

# 2. Обучение

Обучим модели, используя рассчитанные матрицу cо значениями ***TF-IDF:***

Для начала обучим логистическую регрессию и рассчитаем среднее гармоническое полноты и точности (метрику F1):

In [37]:
%%time
log_reg = LogisticRegressionCV(cv=3, random_state=12345, n_jobs=-1, class_weight = 'balanced')
log_reg.fit(features_train, target_train)
pred_log_reg = log_reg.predict(features_train)
f1_score(target_train, pred_log_reg)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda

CPU times: user 12min 13s, sys: 2min 49s, total: 15min 2s
Wall time: 15min 6s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.925380759283502

In [38]:
pred_log_reg = log_reg.predict(features_test)
f1_score(target_test, pred_log_reg)

0.7572270558518104

На тесте показатель выше заявленного в задании.

Перейдем к модели случайного леса:

In [39]:
rfc = RandomForestClassifier(random_state=12345, class_weight = 'balanced')

In [40]:
param_grid = { 
    'n_estimators': [100, 150, 200],
    'max_depth' : np.arange(5, 16, 5),
    'min_samples_split' : [2, 5, 8]
    }

*Внимание!* Следующая ячейка также выполняется порядка 1 часа:

In [41]:
%%time
cv_rfc = GridSearchCV(rfc, param_grid=param_grid, cv= 3, n_jobs=-1)
cv_rfc.fit(features_train, target_train)
cv_rfc.best_params_

CPU times: user 54min 1s, sys: 8.59 s, total: 54min 10s
Wall time: 54min 28s


{'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 150}

In [42]:
%%time
rfc = RandomForestClassifier(random_state=12345, min_samples_split=2, max_depth=15, n_estimators=200, class_weight = 'balanced')
rfc.fit(features_train, target_train)

CPU times: user 23.4 s, sys: 34.3 ms, total: 23.4 s
Wall time: 23.5 s


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=12345, verbose=0, warm_start=False)

In [43]:
pred_rfc = rfc.predict(features_train)
f1_score(target_train, pred_rfc)

0.38897099437204236

In [44]:
pred_rfc = rfc.predict(features_test)
f1_score(target_test, pred_rfc)

0.37777777777777777

Результат гораздо ниже, чем на линейной модели.

Теперь обучим модели, используя сформированные ***BERT-моделью*** векторы:

Логистическая регрессия:

In [45]:
log_reg.fit(features_tr, target_tr)
pred_log_reg = log_reg.predict(features_val)
f1_score(target_val, pred_log_reg)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda

0.5901639344262295

In [46]:
pred_log_reg = log_reg.predict(features_tt)
f1_score(target_tt, pred_log_reg)

0.6666666666666667

Случайный лес:

In [47]:
rfc.fit(features_tr, target_tr)
pred_rfc = rfc.predict(features_val)
f1_score(target_val, pred_rfc)

0.2941176470588235

In [48]:
pred_rfc = rfc.predict(features_tt)
f1_score(target_tt, pred_rfc)

0.27906976744186046

# 3. Выводы

Целью данной работы являлось обучение моделей классификации комментариев на позитивные и негативные. 
Работа состояла из двух этапов: предобработки данных и обучения моделей.
Предобработка данных проводилась двумя способами: 
- при помощи создания мешка слов и оценки частоты употребления слов (TF-IDF и предшествующие величине токенизация и лемматизация);
- посредством векторного представления моделью BERT (с помощью преобразования текстов в эмбеддинги).

После предобработки данных были обучены 2 выбранные модели: логистическая регрессия и случайный лес. Поскольку метрикой качества в данном проекте являлась мера F1, равная по меньшей мере 0.75, этот показатель был взят за минимальное значение.
По итогам обучения и предсказания моделей можно сказать, что показатель свыше 0.75 был достигнут только логистической регрессией и только при подготовке данных при помощи величины TF-IDF. Разница между результатами предсказания на валидационной и тестовой выборке минимальна, поэтому переобучения не произошло.  
Соответственно, логистическая регрессия - та модель, которую стоит выбрать в данном случае.  
Тем не менее, не стоит упускать тот факт, что модель BERT работала лишь с частью данных. Результат получлся весьма неожиданным, и тем не менее, есть предположение, что при наличии больших мощностей, модель BERT предобработала бы данные лучше, и модель также показала бы лучший результат.